In [39]:
import pandas as pd
from pathlib import Path
import scipy
parks_data = pd.read_csv(Path.home() / 'Downloads' / 'poi_with_population_count_2022-01-31.csv')
hps = pd.read_csv(Path.home() / 'Downloads' / '2022-01-26-home_panel_summary.csv')
census = pd.read_csv(Path.home() / 'Downloads' / 'cbg_b01.csv')

In [40]:
print(hps.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271398 entries, 0 to 271397
Data columns (total 7 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   date_range_start                271398 non-null  object
 1   date_range_end                  271398 non-null  object
 2   region                          271398 non-null  object
 3   iso_country_code                271398 non-null  object
 4   census_block_group              271398 non-null  object
 5   number_devices_residing         271398 non-null  int64 
 6   number_devices_primary_daytime  271398 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 14.5+ MB
None


In [41]:
print(parks_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83870 entries, 0 to 83869
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 83870 non-null  int64  
 1   placekey                   83870 non-null  object 
 2   parent_placekey            5182 non-null   object 
 3   location_name              83870 non-null  object 
 4   street_address             83870 non-null  object 
 5   city                       83870 non-null  object 
 6   region                     83870 non-null  object 
 7   postal_code                83870 non-null  int64  
 8   iso_country_code           83870 non-null  object 
 9   safegraph_brand_ids        10183 non-null  object 
 10  brands                     10183 non-null  object 
 11  date_range_start           83870 non-null  object 
 12  date_range_end             83870 non-null  object 
 13  raw_visit_counts           83870 non-null  int

In [42]:
print(census.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220333 entries, 0 to 220332
Columns: 161 entries, census_block_group to B01003m1
dtypes: float64(60), int64(101)
memory usage: 270.6 MB
None


In [43]:
census_first = census[['census_block_group', 'B01001e1']]
census_first = census_first.loc[census_first['census_block_group'].astype(str).str.startswith('36')]
print(census_first.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15463 entries, 127690 to 143152
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   census_block_group  15463 non-null  int64
 1   B01001e1            15463 non-null  int64
dtypes: int64(2)
memory usage: 362.4 KB
None


In [44]:
pop_nys = census_first['B01001e1'].sum()
print(pop_nys)

19572319


In [45]:
#find state devices

hps_ny = hps.loc[hps['region'] == 'ny']
devices_nys = hps_ny['number_devices_residing'].sum()
print(devices_nys)

852188


In [54]:
#find statewide multiplier

mult = pop_nys / devices_nys
print(mult)

22.967137533032616


In [55]:
compare_df = pd.DataFrame()
compare_df['raw_visitor_counts']= parks_data['raw_visitor_counts']
compare_df['state_normalized'] = compare_df['raw_visitor_counts'] * mult
compare_df['cbg_normalized'] = parks_data['visitors_pop_calc']

In [56]:
compare_df['error'] = (compare_df['cbg_normalized'] - compare_df['state_normalized']).apply(abs)
compare_df['percent_error'] = (compare_df['cbg_normalized'] - compare_df['state_normalized']).apply(abs) / compare_df['cbg_normalized']
print(f"mean error: {compare_df['error'].mean()}")
print(f"mean percent error: {compare_df['percent_error'].mean()}")
print(f"avg state normalized: {compare_df['state_normalized'].mean()}")
print(f"avg cbg normalized: {compare_df['cbg_normalized'].mean()}")


mean error: 234.26722517199738
mean percent error: 0.4597828229123951
avg state normalized: 665.4278314533815
avg cbg normalized: 679.9609107841809
